In [ ]:

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tbparse import SummaryReader

# --- Configuration ---
LOG_DIR = "../logs"  
RESULTS_DIR = "../results"
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Loads TensorBoard logs ---
print Loading TensorBoard event files...")
folders = glob.glob(os.path.join(LOG_DIR, "*_*"))

all_data = []
for f in folders:
    try:
        algo = "PPO" if "ppo" in f.lower() else "A2C"
        persona = "explorer" if "explorer" in f.lower() else "survivor"
        reader = SummaryReader(f)
        scalars = reader.scalars
        scalars["algo"] = algo
        scalars["persona"] = persona
        all_data.append(scalars)
    except Exception as e:
        print(f" Skipping {f}: {e}")

if not all_data:
    raise RuntimeError("No valid TensorBoard event logs found.")

# Merges all runs
df = pd.concat(all_data, ignore_index=True)
print(f"Loaded {len(df)} total entries from {len(folders)} runs.")

# --- Filters and renames tags for clarity ---
tags = 
{
    'rollout/ep_rew_mean': 'Mean Episode Reward',
    'rollout/ep_len_mean': 'Mean Episode Length',
    'train/value_loss': 'Value Loss',
    'train/policy_gradient_loss': 'Policy Gradient Loss'
}
df = df[df['tag'].isin(tags.keys())]
df['Metric'] = df['tag'].map(tags)

# --- Plot learning curves ---
plt.figure(figsize=(9,6))
sns.lineplot(data=df[df['Metric']=="Mean Episode Reward"], x='step', y='value', hue='algo', style='persona', linewidth=2)
plt.title("Learning Curves: PPO vs A2C across Personas")
plt.xlabel("Timesteps")
plt.ylabel("Mean Episode Reward")
plt.grid()
plt.legend(title="Algo / Persona")
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "learning_curves.png"), dpi=300)
plt.show()

# --- Value Loss ---
plt.figure(figsize=(9,6))
sns.lineplot(data=df[df['Metric']=="Value Loss"], x='step', y='value', hue='algo', style='persona', linewidth=2)
plt.title("Value Loss over Time")
plt.xlabel("Timesteps")
plt.ylabel("Loss")
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "value_loss.png"), dpi=300)
plt.show()

# --- Persona Comparison ---
plt.figure(figsize=(9,6))
sns.lineplot(data=df[df['Metric']=="Mean Episode Reward"], x='step', y='value', hue='persona', style='algo', linewidth=2)
plt.title("Persona Comparison: Explorer vs Survivor")
plt.xlabel("Timesteps")
plt.ylabel("Mean Reward")
plt.grid()
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, "persona_comparison.png"), dpi=300)
plt.show()

# --- Summary Statistics ---
summary = df.groupby(['algo', 'persona', 'Metric'])['value'].describe()[['mean','std','max']]
summary.to_csv(os.path.join(RESULTS_DIR, "metric_summary.csv"))
print("Summary saved to results/metric_summary.csv")
print(summary.head())
